TOPIC MODELLING PARA review_comments , separados por review_score

Tipo de modelo de ML UNSUPERVISED

VERSION 2, CON SUBCLASE DE COUNTVECTORIZER, CON STEMMER INLCUIDO, Y LDA.

In [33]:

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import make_pipeline
#
import nltk
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
nltk.download('stopwords')
import Stemmer
#Para traducir la salida: 
from deep_translator import GoogleTranslator
#import matplotlib as plt
import pandas as pd
import matplotlib.pyplot as plt

#hay varios q no se usan, del otro archivo. limpiar. 


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/moncho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
# preparamos los datos que usamos:
Review = pd.read_csv('Datasets/olist_order_reviews_dataset.csv')

# tiro nulos: 
Review.dropna(subset = 'review_comment_message', inplace= True)

# me quedo solo con estas cols: TMB AGREGO FECHA A VER SI SE PUEDE PLOTTEAR
Review = Review[['review_score', 'review_comment_message', 'review_id','review_creation_date']]

# los separo por score, para poder sacarle los 'topic' por separado a cada uno.
estrella_1 = Review[Review['review_score'] == 1]
estrella_2 = Review[Review['review_score'] == 2]
estrella_3 = Review[Review['review_score'] == 3]
estrella_4 = Review[Review['review_score'] == 4]
estrella_5 = Review[Review['review_score'] == 5]

In [27]:
#  DEFINO LA SUB CLASE DE CountVectorizer:  

stemmer = Stemmer.Stemmer('pt')  #import Stemmer , ya lo hice en otra celda. 

#stemmer=RSLPStemmer()
analyzer = CountVectorizer().build_analyzer()

# Override TfidfVectorizer
class StemmedCountVectorizer(CountVectorizer):
    '''def Busco_palabra_raiz(texto): #esta esta era la funcion nuestra del ETL del modelo classifier. 
        stemmer=RSLPStemmer()
        return [stemmer.stem(c) for c in texto.split()]'''
    def build_analyzer(self):   #pruebo con esta a ver si me hace un mejor analisis. 
        analyzer = super(CountVectorizer, self).build_analyzer()
        return lambda doc: stemmer.stemWords(analyzer(doc))
    


In [28]:
#stopwords en portugues: EDITAR SEGUN OUTPUTS. 


stoplist = stopwords.words('portuguese')
#le agrego algunas: 
stoplist_ampliada = stopwords.words('portuguese') + ['produto', 'satisfeita', 'n','recomendo', 'recom']
# VER: agregar el 'aun' (en pt)?

#veo de quitarle el 'no', asi arma mas frases con sentido de negacion...
stoplist_ampliada.remove('não')

In [44]:
#por las dudas q se haya transformado lo instacio de nuevo los datos: 
comments1=estrella_1['review_comment_message'] # ie, todos los comentarios de review_score==1. 

# Sacar non-word characters, so numbers and ___ etc: VER SI HACE FALTA ESTA LIMPIEZA. 
#comments1 = comments1.str.replace("[^A-Za-z ]", " ")

#el vectorizador con sus parametros, ACA ES DONDE SE PUEDE JUGAR: 
# Document frequency: min_df=5 (debe aparecer al menos 5 veces), max_df=0.5 (debe aparecer en menos de la mitad)
# agregar max_features=500????
vectorizer = StemmedCountVectorizer(stop_words=stoplist_ampliada, ngram_range=(2,4), min_df=5, max_df=0.5)
matrix = vectorizer.fit_transform(comments1)

#convierte a df la matriz vectorizda. 
words_df = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names())

In [45]:

# Uso LDA: 
# cuantos topics: 
n_topics = 3

model = LatentDirichletAllocation(n_components=n_topics)
# fit con el output de la celda anterior:
model.fit(matrix)

# cuantas words / topic (incluyendo los ngrams...):
n_words = 3
feature_names = vectorizer.get_feature_names()

#TRADUCTOR: instancio uno, si no lo hubiera:
traductor = GoogleTranslator(source='pt', target='es')

topic_list = []
for topic_idx, topic in enumerate(model.components_):
    top_n = [traductor.translate(feature_names[i])   #traductor.translate(feature_names[i])   #lo traduzco asi entiendo...
             for i in topic.argsort()
             [-n_words:]][::-1]
    top_features = ', '.join(top_n) #separo por comas asi se identifican los ngrams de las palabras sueltas.
    topic_list.append(f"topic_{'_'.join(top_n[:3])}") 

    print(f"Topic {topic_idx}: {top_features}")

/Users/moncho/Library/Python/3.9/lib/python/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic 0: no entregado, ningún momento, no me gusta
Topic 1: no recibí, todavía en, no llegues
Topic 2: no recibí, nota fiscal, voltios de dinero


In [ ]:
#output con (2,4), max_df=0.5  
'''Topic 0: nota fiscal, acabo de recibir, voltios de dinero
Topic 1: no llegues, todavía en, no ha llegado aún
Topic 2: no recibí, no entregado, no recomiendo
Topic 3: no recibí, todavía en, no recibido aún
Topic 4: no entregado, todavía en, el tiempo de entrega
Topic 5: No puedo, no funciona, no compre'''
# idem pero le saque el recom o recomendo...
'''Topic 0: no recibí, todavía en, no recibido aún
Topic 1: no recibí, no recomiendo, voltios de dinero
Topic 2: no recibí, no recibí, ningún momento
Topic 3: no funciona, no me gusta, vino defectuoso
Topic 4: acabo de recibir, comprado en, nota fiscal
Topic 5: no llegues, no entregado, todavía en'''
#(3,4), 5 topic, 1 palabra. 
'''Topic 0: no ha llegado aún
Topic 1: no recibido aún
Topic 2: no recibí
Topic 3: no recibido aún
Topic 4: ahora no recibo'''

#2,4  10 topic 1 word
Topic 0: no recibí
Topic 1: no recibí
Topic 2: no recibí
Topic 3: nota fiscal
Topic 4: acabo de recibir
Topic 5: no es original
Topic 6: no recibí
Topic 7: voltios de dinero
Topic 8: no me gusta
Topic 9: no entregado

# idem pero 3,4  
Topic 0: no recibido aún
Topic 1: No hasta ahora
Topic 2: momento no recibido
Topic 3: no recibido aún
Topic 4: ahora no recibo
Topic 5: no recibi la compra
Topic 6: aun no lo he recibido
Topic 7: no ha llegado aún
Topic 8: no obtuve ninguno
Topic 9: no recibí

#idem pero 10 words, 1 topic. 
Topic 0: no recibido aún, no ha llegado aún, ahora no recibo, momento no recibido, todavía no entregado, quiero dinero de vuelta, no recibí, aun no lo he recibido, no recibido aún, no obtuve ninguno

# (2,4), 3, 3. (comments1)
Topic 0: no entregado, ningún momento, no me gusta
Topic 1: no recibí, todavía en, no llegues
Topic 2: no recibí, nota fiscal, voltios de dinero

In [53]:
Review['review_creation_date'].min()

'2016-10-02 00:00:00'

In [54]:
Review['review_creation_date'].max()

'2018-08-31 00:00:00'

In [57]:
# PRUEBO CON LAS DE ULTIMOS 2 MESES. 

estrella_1_ultimos = Review[(Review['review_score'] == 1) & ( Review['review_creation_date']>'2018-06-31 00:00:00')]
#estrella_2 = Review[Review['review_score'] == 2]
#estrella_3 = Review[Review['review_score'] == 3]
#estrella_4 = Review[Review['review_score'] == 4]
estrella_5_ultimos = Review[(Review['review_score'] == 5) & ( Review['review_creation_date']>'2018-06-31 00:00:00')]

In [60]:


#comments1_ultimos=estrella_1_ultimos['review_comment_message'] # me quedo con los comentarios. 
comments5_ultimos=estrella_5_ultimos['review_comment_message'] # me quedo con los comentarios. 

# Sacar non-word characters, so numbers and ___ etc: VER SI HACE FALTA ESTA LIMPIEZA. 
#comments1 = comments1.str.replace("[^A-Za-z ]", " ")

#el vectorizador con sus parametros, ACA ES DONDE SE PUEDE JUGAR: 
# Document frequency: min_df=5 (debe aparecer al menos 5 veces), max_df=0.5 (debe aparecer en menos de la mitad)
# agregar max_features=500????
vectorizer = StemmedCountVectorizer(stop_words=stoplist_ampliada, ngram_range=(2,4), min_df=5, max_df=0.5)
matrix = vectorizer.fit_transform(comments5_ultimos)

#convierte a df la matriz vectorizda. 
words_df = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names())


# Uso LDA: 
# cuantos topics: 
n_topics = 3

model = LatentDirichletAllocation(n_components=n_topics)
# fit con el output de la celda anterior:
model.fit(matrix)

# cuantas words / topic (incluyendo los ngrams...):
n_words = 1
feature_names = vectorizer.get_feature_names()

#TRADUCTOR: instancio uno, si no lo hubiera:
traductor = GoogleTranslator(source='pt', target='es')

topic_list = []
for topic_idx, topic in enumerate(model.components_):
    top_n = [traductor.translate(feature_names[i])   #traductor.translate(feature_names[i])   #lo traduzco asi entiendo...
             for i in topic.argsort()
             [-n_words:]][::-1]
    top_features = ', '.join(top_n) #separo por comas asi se identifican los ngrams de las palabras sueltas.
    topic_list.append(f"topic_{'_'.join(top_n[:3])}") 

    print(f"Topic {topic_idx}: {top_features}")

/Users/moncho/Library/Python/3.9/lib/python/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic 0: entrega rápida
Topic 1: antes de la fecha límite
Topic 2: bien antes


PARA comments1_ultimo, (2,4), 3,3. 
Topic 0: no recibí, no vino, no funciona
Topic 1: no entregado, no recibí, no recibí
Topic 2: todavía en, no llegues, No puedo
Para comments5_ultimo: 
Topic 0: antes de la fecha límite, entregado, a tiempo
Topic 1: ha llegado, antes de la fecha límite, llegó antes de la fecha límite
Topic 2: antes de la fecha límite, bien antes, mucho antes de la fecha límite

OSEA, NO CAAMBIA MUCHO, INCLUSO SI ANALIZAMOS PARA ULTIMOS DOS MESES, LOS TOPICS SIGUEN SIENDO LOS MISMOS.!!